In [1]:
cd /data/projects/windturbine-identification/MachineLearningCourse

/data/projects/windturbine-identification/MachineLearningCourse


In [2]:
from scripts.windturbines.osm_download import getfeatures, get_param

import overpy

import fiona
from fiona.crs import from_epsg

from shapely.geometry import Point, mapping
import re

import argparse

import math
import time

from pathlib import Path

import os

import pandas as pd

In [3]:
def writeshape(nodes, name):

    schema = { 'geometry': 'Point', 'properties': {'method': 'str', 'source': 'str', 'elmw': 'float'} }
    shapeout = name

    with fiona.open(shapeout, 'w', crs=from_epsg(4326), driver='ESRI Shapefile', schema=schema) as output:
        for node in nodes:
            gensource = node.tags.get("generator:source", "unknown")
            genmethod = node.tags.get("generator:method", "unknown")
            genmodel = node.tags.get("generator:model", "unknown")

            genoutel = node.tags.get("generator:output:electricity", "unknown")
            genoutmw = -9999
            if re.search("[+-]?([0-9]*[.])?[0-9]+", genoutel):
                genoutmw = float(re.search("[+-]?([0-9]*[.])?[0-9]+", genoutel)[0])
                if "kw" in genoutel.lower():
                    genoutmw = genoutmw / 1000
            try:
                point = Point(node.lon, node.lat)
            except AttributeError:
                point = Point(node.center_lon, node.center_lat)
            prop = {
                'method': genmethod.lower(),
                'source': gensource.lower(),
                'elmw': genoutmw}
            output.write({'geometry': mapping(point), 'properties': prop})

In [4]:

countries_full =["EG", "MA", "ZA", "CN", "IN", "JP", "KP", "PK", "PH", "TH", "CR", "TR", "AT", "BE", "BG", "HR", "DK", "FI", 
                  "FR", "DE", "GR", "IE", "IT", "LT", "NL", "NO", "PL", "PT", "RO", "ES", "SE", "GB", "UA", "CA", 
                  "MX", "US", "AU", "NZ", "BR", "CL", "UY"]

countries =["EG", "MA", "ZA", "IN", "JP", "KP", "PK", "PH", "TH", "CR", "TR", "AT", "BE", "BG", "HR", "DK", "FI", 
            "GR", "IE", "IT", "LT", "NL", "NO", "PL", "PT", "RO", "SE", "UA",  
            "MX", "AU", "NZ", "CL", "UY"]

countries = ["US"]



In [5]:
# countries = ['IN', 'ES', 'GB', 'FR', 'BR', 'CA']



#countries = ['FR', 'BR', 'CA']


for country in countries:
    
    print("Working on country: " + country)
    
    st = time.time()
    nodes, ways, rels = getfeatures(country, "power", "plant")
    features = nodes + ways + rels
    et = time.time()

    print('waiting for {:.0f} seconds'.format(et - st))
    time.sleep(et - st)

    print('fetching generators')
    nodes, ways, rels = getfeatures(country, "power", "generator")
    features = features + nodes + ways + rels

    PATH_OSM = get_param(country,"PATH_OSM")

    writeshape(features, PATH_OSM)



Working on country: US
Starting download...
waiting for 12 seconds
waiting for 59 seconds
	nodes: 29, ways: 2226, relations: 0
waiting for 322 seconds
fetching generators
Starting download...
waiting for 188 seconds
waiting for 446 seconds


OverpassTooManyRequests: Too many requests

In [5]:
get_param("EG","PATH_OSM")

'data/osm/EG'